In [1]:

import re
from sklearn.naive_bayes import MultinomialNB
import pandas as pd
import matplotlib.pyplot as plt

from nltk import word_tokenize
import string
string.punctuation

from gensim.parsing.preprocessing import remove_stopwords

import seaborn as sns
import matplotlib.pyplot as plt

import numpy as np

from sklearn.metrics import*

from sklearn.model_selection import train_test_split


/Users/ramyanee/opt/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
df = pd.read_csv('yelp_labelled.txt',sep=".\t",header=None)

<ipython-input-2-8d40498fd0f1>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('yelp_labelled.txt',sep=".\t",header=None)


# Pre-processing 

### Steps: 
#### removing punctuations  
#### removing urls  
#### removing htms tags if any  
#### removing stopwords  
#### converting to lower case 
#### tokenization

In [3]:
def preprocess(text):
    text = re.sub(re.compile('<.*?>') , '', text)
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = text.lower()
    text = "".join([t for t in text if t not in string.punctuation])
    text = remove_stopwords(text)
    text = word_tokenize(text)
    return np.array(text)

def avgw2v(text,dim,model):
    n = len(text)
    sumVector = np.zeros(dim)

    for t in text:
        sumVector = np.add(sumVector,model.wv[t])
    return np.divide(sumVector,n)

# Before Pre-Processing

In [4]:
df[0]

0                                Wow... Loved this place
1                                      Crust is not good
2               Not tasty and the texture was just nasty
3      Stopped by during the late May bank holiday of...
4      The selection on the menu was great and so wer...
                             ...                        
995    I think food should have flavor and texture an...
996                              Appetite instantly gone
997    Overall I was not impressed and would not go back
998    The whole experience was underwhelming, and I ...
999    Then, as if I hadn't wasted enough of my life ...
Name: 0, Length: 1000, dtype: object

# After Pre-Processing

In [5]:
tokenizedReviews=df[0].apply(lambda x:preprocess(x))
tokenizedReviews

0                                    [wow, loved, place]
1                                          [crust, good]
2                                [tasty, texture, nasty]
3      [stopped, late, bank, holiday, rick, steve, re...
4                       [selection, menu, great, prices]
                             ...                        
995              [think, food, flavor, texture, lacking]
996                          [appetite, instantly, gone]
997                                 [overall, impressed]
998    [experience, underwhelming, think, ninja, sush...
999    [hadnt, wasted, life, poured, salt, wound, dra...
Name: 0, Length: 1000, dtype: object

In [6]:
x_train, x_test, y_train, y_test = train_test_split(np.array(tokenizedReviews),df[1], test_size=0.2,stratify=df[1],random_state=42)


# Creating vocabulary of unique words from the training set

In [7]:
words = []
for t in x_train:
    for w in t:
        if w not in words:
            words.append(w)

In [8]:
words.sort()

In [9]:
len(words)

1599

# Word Count feature matrices

In [10]:
TrainMat = np.zeros((len(x_train),len(words)))
for i in range(len(x_train)):
    
    for w in x_train[i]:
        # print(w)
        idx = words.index(w)
        # print(idx)
        TrainMat[i][idx]+=1



In [11]:
TrainMat.shape

(800, 1599)

In [12]:
TestMat = np.zeros((len(x_test),len(words)))
for i in range(len(x_test)):
    
    for w in x_test[i]:
        if w in words:
            idx = words.index(w)
            TestMat[i][idx]+=1

In [13]:
TestMat.shape

(200, 1599)

# Multinomial Naive Bayes with add-1 smoothing

In [14]:
clf = MultinomialNB(alpha=1)
clf.fit(TrainMat,y_train)


MultinomialNB(alpha=1)

# Accuracy and F-1 Scores


#### Validation: Accuracy = 81%, F1-Score = 0.8173076923076923
#### Training: Accuracy = 95.625%, F1-Score = 0.9561952440550688

In [19]:
clf.score(TestMat,y_test)

0.81

In [20]:
clf.score(TrainMat,y_train)

0.95625

In [21]:
f1_score(y_test, clf.predict(TestMat))

0.8173076923076923

In [22]:
f1_score(y_train, clf.predict(TrainMat))

0.9561952440550688